## Libraries I used

In [1]:
import httplib
import urllib2
import random
import re
import requests
from httplib import IncompleteRead
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sys  
reload(sys)  
sys.setdefaultencoding('latin-1')

## List "arrondissement" & "departement" & newpapers of Senegal

In [2]:
arrondissement = ['Almadies','Agnam Civol','Baba Garage','Bandafassi','Brassou','Bala','Bélé','Boghal','Bona',
                  'Boyngel Bamba','Bamba Thialène','Barkédji','Bembou','Cas-Cas','Colobane','Coki',
                  'Darou Minam','Dar Salam','Dakateli','Dagoudane','Dodji','Dakar Plateau',
                  'Diakhao','Djilor','Djibanar','Djoulacolon','Diaroumé','Darou Mousty','Dianké Makha','Diendé',
                  'Djibabouya','Djiredji','Fafacourou','Fissel','Fongolembi', 'Fimela',
                  'Grand Dakar','Gamadji Saré','Guédiawaye','Gniby',
                  'Kataba','Kael','Kabrousse','Katakel','Karatanba','Koussanar','Keur Moussa',
                  'Kéniaba','Koulor','Koumbal','Kouthiaba Wolof','Keur Momar Sarr','Keur Mboucki',
                  'Lambaye','Lour Escale','Loudia Ouoloff','Ida Mouride',
                  'Mampatim','Makacolibantang','Moudéri','Mbane','Mbadakhoune','Missira Wadine','Missirah','Mbédiène',
                  'Ndindy','Ndoulo','Ndiédieng','Ndande','Ndame','Niodor','Ngoye','Ndorna',
                  'Niaming','Niayes','Ndiaye','Niakhar','Notto','Mabo','Médina Sabakh',
                  'Ogo','Orkadiere','Ouadiour', 'Parcelles Assainies', 'Paoskoto','Rao','Rufisque',
                  'Sagatta','Sagatta Dioloff','Saré Bidji','Sindian','Sangalkam','Sagna','Sakal',
                  'Sabodala','Sibassor','Sindia','Séssène','Saldé','Thiénaba','Thiès',
                  'Taif','Tendouck','Tenghory','Tattaguine','Thiaroye','Toubacouta','Thillé Boubacar',
                  'Vélingara','Wouro Sidy','Wack Ngouna','Yang Yang']


departement=['Bakel','Bambey','Bignona','Birkelane','Bounkiling','Dagana','Dakar','Diourbel','Fatick','Foundiougne',
            'Gossas','Goudiry','Goudomp','Guédiawaye','Guinguinéo','Kaffrine','Kanel','Kaolack',
             'Kébémer','Kédougou','Kolda','Koumpentoum','Koungheul','Linguère','Louga','Mbour',
             'Malem Hodar','Matam','Mbacké','Médina Yoro Foulah','Nioro du Rip','Oussouye',
             'Pikine','Podor','Ranérou-Ferlo','Rufisque','Saint-Louis','Salemata','Saraya',
             'Sédhiou','Tambacounda','Thiès','Tivaouane','Vélingara','Ziguinchor']

site_web=["http://www.aps.sn","http://www.walf-groupe.com","http://www.sudonline.sn",
          "http://www.lesoleil.sn","http://www.lequotidien.sn","http://xalimasn.com",
          "http://www.seneweb.com/index.php","http://www.lagazette.sn",
          "http://www.senetoile.net","http://senego.com",
          "http://www.sen360.com","http://www.senenews.com",
          "http://www.sen24heures.com",
          "http://www.nettali.net","http://www.leuksenegal.com",
          "http://www.leral.net","http://www.au-senegal.com/?lang=fr",
          "http://www.dakaractu.com","http://actusen.com",
          "http://www.devoircitoyen.fr",
          "http://www.homeviewsenegal.com","http://www.enqueteplus.com",
          "http://www.gouv.sn","http://www.jaaduwul.com","http://www.sn.undp.org",
          "http://www.senegaltribune.com/articles","http://www.slateafrique.com",
          "http://www.caritas.sn","http://senegal-business.com",
          "http://xibaaru.com","http://www.xibar.net","http://www.lifixew.com",
          "http://annagueye.blogspot.com","http://www.ferloo.com",
          "http://www.derniereminute.sn","http://www.sunuker.com","http://laviesenegalaise.net",
          "http://mathiamenvironmentsenegal.blogspot.com","http://lasomone.com",
          "http://ledakarois.net","http://www.dakar-echo.com","http://www.telesenegal.com",
          "http://www.igfm.sn","http://galsen221.com","http://www.dakarposte.com",
          "http://mediafrik.com","http://www.planete-senegal.com","http://allodakar.com",
          "http://www.ndarinfo.com"]

## Interation1 : loop thru newspapers and extract all links

In [3]:
all_page=[]
for i,s in enumerate(site_web):
    print i,s
    try:
        req = urllib2.Request(s.decode('utf-8'), headers={'User-Agent' : "Magic Browser"}) 
        page = urllib2.urlopen( req )
        soup = BeautifulSoup(page)
    except urllib2.HTTPError,e:
        continue
    except urllib2.URLError, e:
        continue
    except httplib.HTTPException, e:
        continue
    except Exception:
        continue
    #find all links
    all_links= soup.find_all("a")
    #ensure all links are valid
    link_inside= [link.get("href") for link in all_links if link.get("href") is not None]
    link_inside= [link if link.startswith('http') or link.startswith('www') else s + link for link in link_inside]
    #do not keep links from starter links
    link_inside = [link for link in link_inside if link not in site_web]
    #do not keep links that have been already recorded from the current loop
    link_inside = [link for link in link_inside if link not in all_page]
    for k in np.unique(link_inside):
        all_page.append(k)

###  Keep unique obs

In [4]:
all_page=np.unique(all_page)
len(all_page)

15428

### Save "all_page" variable in csv

In [5]:
df = pd.DataFrame(data=all_page, index= range(len(all_page)), columns=['link'])
df.to_csv("all_page.csv",sep=";",encoding="utf-8")

## Iteration 2: loop again from the obtained links to add more depth and attain archives

In [6]:
all_page1=[]

## randomly sample "all_page" to introduce randomness

In [7]:
all_page= random.sample(all_page,len(all_page))

In [ ]:
for i,s in enumerate(all_page[1580:]):
    print i, s
    try:
        if "pdf" not in s and "jpg" not in s and "javascript" not in s:
            req = urllib2.Request(s.decode('utf-8'), headers={'User-Agent' : "Magic Browser"})
            page = urllib2.urlopen( req )
        else:
            continue
    except urllib2.HTTPError,e:
        continue
    except urllib2.URLError, e:
        continue
    except httplib.HTTPException, e:
        continue
    except Exception:
        continue
    #handle page that does not actually exist
    if len(page.read())==0:
        print "the above page does not exist"
        continue
    else:
        soup = BeautifulSoup(page)
        #find all links from the current "page"
        all_links= soup.find_all("a")
        #ensure all links are valid
        link_inside= [link.get("href") for link in all_links if link.get("href") is not None]
        link_inside= [link if link.startswith('http') or link.startswith('www') else s + link for link in link_inside]
        #do not append links from starter links
        link_inside = [link for link in link_inside if link not in site_web]
        #do not append links from previous records
        link_inside = [link for link in link_inside if link not in all_page]
        #do not append links in the current records
        link_inside = [link for link in link_inside if link not in all_page1]
        for k in np.unique(link_inside):
            print k, len(all_page1)
            all_page1.append(k)

### Keep unique pages

In [25]:
all_page1=np.unique(all_page1)
all_page1=list(all_page1)
len(all_page1)

73016

### Filter to remove social media links

In [26]:
f=filter(lambda x: 'linkedin' in x or 'facebook' in x or 'twitter' in x or 'youtube' in x or 'http://www.meteo-senegal.net' in x, all_page1)
for k in site_web:
    f.append(k)
all_page1= [x for x in all_page1 if x not in f]
len(all_page1)

72894

### Build and save a dataframe with "all_page1" variable

In [27]:
df = pd.DataFrame(data=all_page1, index= range(len(all_page1)), columns=['link'])
df.to_csv("base_links.csv",sep=";",encoding="utf-8")
df.head()

,link
0,http://311.inburlingtonvt.us/index.php?title=9...
1,http://50.undp.org
2,http://50.undp.org/en/#future
3,http://50.undp.org/en/#timeline
4,http://50.undp.org/en/africa


### Search for keywords related to flood events

In [29]:
flood_kwds='(pluviométri|bidonville|montée des eaux|flood|zone risque|remblai|effondrement|engorgement|débordé|digue|submergé|éclatement barrage|'
flood_kwds= flood_kwds+'rupture barrage|glissement terrain|climatique|inondé|inondation|inondations|pluies|pluie)'
#2nd iter
flood_page_from_all_page1= [x for x in all_page1 if re.search(flood_kwds,x) is not None]
#1st iter
flood_page_from_all_page= [x for x in all_page if re.search(flood_kwds,x) is not None]
flood_page=flood_page_from_all_page + flood_page_from_all_page1
flood_page=np.unique(flood_page)
flood_page

array([ u'http://del.icio.us/post?url=http://www.sunuker.com/2015/05/26/video-quand-le-parapluie-du-gentleman-obama-embrase-les-reseaux-sociaux/&title=VIDEO \u2013 Quand le parapluie du \xab\xa0gentleman\xa0\xbb Obama embrase les r\xe9seaux sociaux',
       u'http://digg.com/submit?phase=2&url=http://www.sunuker.com/2015/05/26/video-quand-le-parapluie-du-gentleman-obama-embrase-les-reseaux-sociaux/&title=VIDEO \u2013 Quand le parapluie du \xab\xa0gentleman\xa0\xbb Obama embrase les r\xe9seaux sociaux',
       u'http://laviesenegalaise.com/inondations-saint-louis-eaux-video',
       u'http://laviesenegalaise.com/inondations-saint-louis-eaux-video#respond',
       'http://ledakarois.netbethio-radie-ses-cheikhs-apres-le-ndigueul-du-khalife.html',
       u'http://sphinn.com/submit.php?url=http://www.sunuker.com/2015/05/26/video-quand-le-parapluie-du-gentleman-obama-embrase-les-reseaux-sociaux/&title=VIDEO \u2013 Quand le parapluie du \xab\xa0gentleman\xa0\xbb Obama embrase les r\xe9seaux s

## Save results:

In [30]:
df = pd.DataFrame(data=flood_page, index= range(len(flood_page)), columns=['link'])
df.to_csv("potential_flood_events.csv",sep=";",encoding="utf-8")
df.head()

,link
0,http://del.icio.us/post?url=http://www.sunuker...
1,http://digg.com/submit?phase=2&url=http://www....
2,http://laviesenegalaise.com/inondations-saint-...
3,http://laviesenegalaise.com/inondations-saint-...
4,http://ledakarois.netbethio-radie-ses-cheikhs-...


In [31]:
df.shape

(139, 1)

# Output: link of page, story,location and date 